In [7]:
import h2o
import pandas as pd
import src.dataCleaning as dc
from h2o.automl import H2OAutoML

In [8]:
diamonds =  pd.read_csv('./diamonds-datamad1019/data.csv/data.csv')

In [9]:
diamonds = dc.dfCleaning(diamonds)

In [10]:
diamonds.head()

,carat,depth,table,price,color_num,cut_num,clarity_num,volume
0,2.26,61.9,57.0,12831,4.0,5.0,2.0,366.903680
1,2.43,63.2,57.0,16170,3.0,3.0,2.0,392.176400
2,0.80,61.0,57.0,2797,5.0,4.0,2.0,133.001901
3,0.40,63.3,60.0,630,5.0,5.0,1.0,64.059840
4,0.31,61.6,55.0,698,4.0,5.0,4.0,51.797610


In [11]:
h2o.init(nthreads = -1, max_mem_size = 26)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.4" 2019-07-16; OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3); OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)
  Starting server from /home/mariobru/.local/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpi_r_odt6
  JVM stdout: /tmp/tmpi_r_odt6/h2o_mariobru_started_from_python.out
  JVM stderr: /tmp/tmpi_r_odt6/h2o_mariobru_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Europe/Madrid
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.11
H2O cluster version age:,4 days
H2O cluster name:,H2O_from_python_mariobru_vab36l
H2O cluster total nodes:,1
H2O cluster free memory:,26 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"accepting new members, healthy"


In [12]:
diamonds_h2o=h2o.H2OFrame(diamonds)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [13]:
diamonds_h2o.head()

carat,depth,table,price,color_num,cut_num,clarity_num,volume
2.26,61.9,57,12831,4,5,2,366.904
2.43,63.2,57,16170,3,3,2,392.176
0.8,61,57,2797,5,4,2,133.002
0.4,63.3,60,630,5,5,1,64.0598
0.31,61.6,55,698,4,5,4,51.7976
0.53,60.5,60,1132,5,3,2,85.5027
0.7,60.8,61,1987,7,4,2,110.716
1.13,64,58,3952,5,2,2,179.758
0.35,60.9,55,614,6,5,3,57.3448
1.01,61.4,66,5062,3,1,5,153.103


In [14]:
y_columns = "price"
x_columns = ["carat","color_num","cut_num","clarity_num","depth"]

In [15]:
train, test=diamonds_h2o.split_frame(ratios = [.999])
X_train=train[x_columns]
y_train=train[y_columns]
X_test=test[x_columns]
y_test=test[y_columns]

In [16]:
aml_ti = H2OAutoML(max_runtime_secs= 360,max_models= 15, seed= 1, nfolds=0)
aml_ti.train(x = x_columns, y = y_columns, training_frame = train, validation_frame=test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [17]:
lb_ti = aml_ti.leaderboard
lb_ti

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_2_AutoML_20191210_092347,289686,538.225,289686,285.346,0.108818
GBM_1_AutoML_20191210_092347,293169,541.451,293169,290.592,nan
XGBoost_3_AutoML_20191210_092347,297704,545.623,297704,288.898,nan
GBM_3_AutoML_20191210_092347,299968,547.694,299968,292.485,nan
XGBoost_1_AutoML_20191210_092347,305533,552.75,305533,285.716,0.109701
GBM_4_AutoML_20191210_092347,311144,557.803,311144,307.018,nan
XGBoost_2_AutoML_20191210_092347,314136,560.478,314136,305.406,nan
GBM_5_AutoML_20191210_092347,322578,567.959,322578,294.382,0.116042
XGBoost_grid_1_AutoML_20191210_092347_model_1,361185,600.987,361185,298.426,0.112953
DeepLearning_1_AutoML_20191210_092347,403429,635.161,403429,351.485,0.169915


In [18]:
pred_automl = aml_ti.leader.predict(test)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [19]:
pred_automl

predict
840.682
925.66
3650.49
1063.98
472.023
1991.76
3432.91
9318.95
5024.09
2225.81


In [20]:
y_test

price
700
1040
4183
1003
427
2016
3748
9576
5345
2390


In [21]:
#Probamos con test

In [22]:
diamonds_test =  pd.read_csv('./diamonds-datamad1019/test.csv')

In [23]:
diamonds_test = dc.dfCleaning(diamonds_test)

In [24]:
diamonds_h2o=h2o.H2OFrame(diamonds_test)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [25]:
x_columns = ["carat","color_num","cut_num","clarity_num","depth"]

In [26]:
X_train=diamonds_h2o[x_columns]

In [27]:
pred_automl = aml_ti.leader.predict(X_train)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [28]:
pred_automl

predict
1238.98
6634.99
1657.63
3826.08
1780.48
10059.5
794.851
866.099
671.164
13861.5


In [29]:
submission = pred_automl.as_data_frame()

In [30]:
submission

,predict
0,1238.980816
1,6634.991527
2,1657.629506
3,3826.082592
4,1780.480069
...,...
13480,2183.086762
13481,2031.353158
13482,11229.312266
13483,518.454449


In [31]:
submission.rename(columns={"predict": "price"}, inplace=True)
submission

,price
0,1238.980816
1,6634.991527
2,1657.629506
3,3826.082592
4,1780.480069
...,...
13480,2183.086762
13481,2031.353158
13482,11229.312266
13483,518.454449


In [32]:
submission.price = submission.price.astype(int)


In [33]:
submission

,price
0,1238
1,6634
2,1657
3,3826
4,1780
...,...
13480,2183
13481,2031
13482,11229
13483,518


In [34]:
submission.to_csv('./diamonds-datamad1019/submission.csv')